---
title: "Precio del USDT vs BOB"
authors:
  - Andres Chirinos
  - Kevin Barrientos
lang: es
format: 
  dashboard:
    theme: cosmo
---

In [9]:
import pandas as pd
import plotly.graph_objects as go
import pytz
from plotly.offline import init_notebook_mode, iplot

# Inicializar el modo de notebook
init_notebook_mode(connected=True)

# Load the data
trade_df = pd.read_csv("usdt_bob_prices.csv")

# Verificar las columnas presentes en el DataFrame
#print(trade_df.columns)

# Asegurarse de que las columnas necesarias estén presentes
required_columns = ["index", "timestamp", "price", "tradeType", "monthOrderCount", "tradableQuantity"]
for col in required_columns:
    if col not in trade_df.columns:
        raise ValueError(f"Missing required column: {col}")

# Set the index
trade_df = trade_df.set_index("index")

# Filter the data
trade_df = trade_df.query("monthOrderCount >= 100")

# Separate buy and sell data
buy_df = trade_df.query("tradeType == 'BUY'")
sell_df = trade_df.query("tradeType == 'SELL'")

# Convertir el índice de timestamp a fechas en la zona horaria GMT-4
gmt_minus_4 = pytz.timezone('Etc/GMT-4')
buy_df['timestamp'] = pd.to_datetime(buy_df['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert(gmt_minus_4)
sell_df['timestamp'] = pd.to_datetime(sell_df['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert(gmt_minus_4)

# Calculate daily OHLC (Open, High, Low, Close) prices and volume for buy and sell
buy_ohlc_df = buy_df.groupby(buy_df['timestamp'].dt.date).agg(
    open=('price', 'first'),
    high=('price', 'max'),
    low=('price', 'min'),
    close=('price', 'last'),
    volume=('tradableQuantity', 'sum')
).reset_index()

sell_ohlc_df = sell_df.groupby(sell_df['timestamp'].dt.date).agg(
    open=('price', 'first'),
    high=('price', 'max'),
    low=('price', 'min'),
    close=('price', 'last'),
    volume=('tradableQuantity', 'sum')
).reset_index()

C:\Users\Andres\AppData\Local\Temp\ipykernel_12492\2161102906.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Andres\AppData\Local\Temp\ipykernel_12492\2161102906.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Row{height=20%}

In [10]:
#| content: valuebox
#| title: "Precio de Compra de Hoy"
#| icon: currency-dollar
#| color: green
dict(
  value = buy_ohlc_df["open"].iloc[-1]
)

{'value': 10.57}

In [15]:
#| content: valuebox
#| title: "Precio de Venta de Hoy"
#| icon: currency-dollar
#| color: red
dict(
  value = sell_ohlc_df["open"].iloc[-1]
)

{'value': 10.61}

In [13]:
#| content: valuebox
#| title: "Volumen de Hoy"
#| icon: currency-dollar
#| color: secondary
dict(
  value = buy_ohlc_df["volume"].iloc[-1]
)

{'value': 2176689.39}

## Row{height=80%}

In [11]:
# Crear el gráfico de velas para precios de compra y venta
fig = go.Figure()

# Añadir el gráfico de velas para compra
fig.add_trace(go.Candlestick(
    x=buy_ohlc_df['timestamp'],
    open=buy_ohlc_df['open'],
    high=buy_ohlc_df['high'],
    low=buy_ohlc_df['low'],
    close=buy_ohlc_df['close'],
    name='Precio de Compra',
    increasing_line_color='blue',
    decreasing_line_color='blue'
))

# Añadir el gráfico de velas para venta
fig.add_trace(go.Candlestick(
    x=sell_ohlc_df['timestamp'],
    open=sell_ohlc_df['open'],
    high=sell_ohlc_df['high'],
    low=sell_ohlc_df['low'],
    close=sell_ohlc_df['close'],
    name='Precio de Venta',
    increasing_line_color='red',
    decreasing_line_color='red'
))

# Añadir el volumen de compra
fig.add_trace(go.Bar(
    x=buy_ohlc_df['timestamp'],
    y=buy_ohlc_df['volume'],
    name='Volumen de Compra',
    marker_color='blue',
    opacity=0.3,
    yaxis='y2'
))

# Añadir el volumen de venta
fig.add_trace(go.Bar(
    x=sell_ohlc_df['timestamp'],
    y=sell_ohlc_df['volume'],
    name='Volumen de Venta',
    marker_color='red',
    opacity=0.3,
    yaxis='y2'
))

# Añadir una línea fija en el valor 6.96
fig.add_shape(
    type="line",
    x0=buy_ohlc_df['timestamp'].min(),
    y0=6.96,
    x1=buy_ohlc_df['timestamp'].max(),
    y1=6.96,
    line=dict(
        color="green",
        width=2,
        dash="dash",
    ),
    name='Línea Fija 6.96'
)

# Configurar etiquetas y título
fig.update_layout(
    title='Precio de Compra y Venta a lo Largo del Tiempo',
    xaxis_title='Fecha',
    yaxis_title='Precio',
    autosize=True,
    height=600,
    yaxis2=dict(
        title='Volumen',
        overlaying='y',
        side='right'
    )
)

# Mostrar el gráfico
iplot(fig)